In [1]:
# Parameters
cell_type = "Plasma"


In [2]:
import sys
import os

import anndata as ad

import pandas as pd
import numpy as np

from pyprojroot import here

import re

from tqdm import tqdm

In [3]:
def load_sorted_shap_values_fnames(
    cell_type: str = '',
    run_name: str = 'run1'):

    dirpath = here(f"inflammabucket_bkp/03_downstream_analysis/05_SHAP/results/04_shap/shap_vals")
    fname_regex = f'^{run_name}_{cell_type}_shap_values_' + r'(\d+)'
    results_batches = [
        (fname, int(re.search(fname_regex, fname).group(1)))
        for fname in os.listdir(dirpath)
        if re.search(fname_regex, fname)]

    sorted_files = sorted(results_batches, key=lambda x: x[1])

    sorted_filenames = [os.path.join(dirpath, filename) for filename, _ in sorted_files]

    return sorted_filenames

In [4]:
shap_val_list = [np.load(fname)['shap_values'] for fname in tqdm(load_sorted_shap_values_fnames(cell_type))] 


  0%|          | 0/2 [00:00<?, ?it/s]

 50%|█████     | 1/2 [00:02<00:02,  2.28s/it]

100%|██████████| 2/2 [00:04<00:00,  2.03s/it]

100%|██████████| 2/2 [00:04<00:00,  2.07s/it]

In [5]:
adata = ad.read_h5ad(here(f'inflammabucket_bkp/03_downstream_analysis/05_SHAP/data/{cell_type}_adataMerged_SPECTRAgenes.log1p.h5ad'), backed='r')

In [6]:
## Gene symbols and Disease labels
symbols_df = pd.read_pickle(here('inflammabucket_bkp/03_downstream_analysis/04_selected_gene_list.pkl'))
symbols_sorted = symbols_df.loc[adata.var_names].symbol.values

DISEASES = ['BRCA', 'CD', 'COPD', 'COVID', 'CRC', 'HBV', 'HIV', 'HNSCC', 'MS', 'NPC', 'PS', 'PSA', 'RA', 'SLE', 'UC', 'asthma', 'cirrhosis', 'flu', 'healthy', 'sepsis']

diseaseDict = dict()
for d in DISEASES:
    diseaseDict[d] = []

In [7]:
shape_values_matrix = np.concatenate(shap_val_list)

In [8]:
for idx, values in tqdm(adata.obs.groupby('sampleID', observed=True).indices.items()):
    geneXdisease_sample_i = pd.DataFrame(shape_values_matrix[values].mean(0))
    geneXdisease_sample_i.columns = DISEASES
    geneXdisease_sample_i.index = symbols_sorted
    for d in geneXdisease_sample_i.columns:
        diseaseDict[d].append(pd.DataFrame.from_dict({idx:geneXdisease_sample_i[d]}))

  0%|          | 0/727 [00:00<?, ?it/s]

  6%|▌         | 42/727 [00:00<00:01, 416.91it/s]

 12%|█▏        | 84/727 [00:00<00:01, 386.38it/s]

 17%|█▋        | 127/727 [00:00<00:01, 404.41it/s]

 23%|██▎       | 169/727 [00:00<00:01, 355.51it/s]

...

 78%|███████▊  | 566/727 [00:01<00:00, 448.45it/s]

 85%|████████▍ | 616/727 [00:01<00:00, 462.95it/s]

 91%|█████████▏| 664/727 [00:01<00:00, 369.15it/s]

 98%|█████████▊| 709/727 [00:01<00:00, 388.15it/s]

100%|██████████| 727/727 [00:01<00:00, 407.49it/s]

In [9]:
for d in tqdm(DISEASES):
    pd.concat(diseaseDict[d], axis=1).to_csv(here(f'03_downstream_analysis/08_gene_importance/new_shap_plots/results/SHAP_AVGsamples/SHAP_AVGsample_{cell_type}_{d}.csv'))

  0%|          | 0/20 [00:00<?, ?it/s]

  5%|▌         | 1/20 [00:00<00:14,  1.33it/s]

 10%|█         | 2/20 [00:01<00:11,  1.57it/s]

 15%|█▌        | 3/20 [00:01<00:11,  1.53it/s]

 20%|██        | 4/20 [00:03<00:13,  1.22it/s]

...

 85%|████████▌ | 17/20 [00:12<00:02,  1.22it/s]

 90%|█████████ | 18/20 [00:13<00:01,  1.13it/s]

 95%|█████████▌| 19/20 [00:14<00:00,  1.06it/s]

100%|██████████| 20/20 [00:15<00:00,  1.03it/s]

100%|██████████| 20/20 [00:15<00:00,  1.30it/s]